In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tweet-sentiment-classification-dataset/tweet_sentiment.csv


In [2]:
df = pd.read_csv('/kaggle/input/tweet-sentiment-classification-dataset/tweet_sentiment.csv')
df

,tweet,sentiment
0,The event starts at 5 PM.,neutral
1,I hate how this turned out.,negative
2,Fantastic experience!,positive
3,Fantastic experience!,positive
4,This is the worst thing ever!,negative
...,...,...
995,It’s cloudy outside.,neutral
996,Great job by the team!,positive
997,I'm so happy about the news!,positive
998,It ruined my whole day.,negative


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet      1000 non-null   object
 1   sentiment  1000 non-null   object
dtypes: object(2)
memory usage: 15.8+ KB


In [4]:
# all unique chars that occure in the tweet
all_text = ''.join(df['tweet'].astype(str))
chars = sorted(set(all_text))
vocab_size=len(chars)
print("Unique Chars")
print(''.join(chars))
print('\nToatl unique charactes:',len(chars))

Unique Chars
 !'.5AFGIJMPRSTWabcdefghijklmnoprstuvwxy’

Toatl unique charactes: 41


In [5]:
# Create a mapping from chars to integere
map1 = { ch:i for i,ch in enumerate(chars) }
map2 = { i:ch for i,ch in enumerate(chars) }

encode = lambda s: [map1[c] for c in s]# encoder: take a string, output a list of integers.
decode = lambda l: ''.join([map2[i] for i in l])# decoder: take a list of integers, output a string

print(encode("Great job by the team!"))
print(decode(encode("Great job by the team!")))

[7, 32, 20, 16, 34, 0, 25, 30, 17, 0, 17, 39, 0, 34, 23, 20, 0, 34, 20, 16, 28, 1]
Great job by the team!


* Let's now encode the entire text dataset and store it into a torch.Tensor

In [6]:
import torch 
data = torch.tensor(encode(all_text),dtype=torch.long)
print(data.shape,data.dtype)
print(data[:1000])# 1000 characters we looked at earier will to the GPT look like this

torch.Size([24608]) torch.int64
tensor([14, 23, 20,  0, 20, 36, 20, 29, 34,  0, 33, 34, 16, 32, 34, 33,  0, 16,
        34,  0,  4,  0, 11, 10,  3,  8,  0, 23, 16, 34, 20,  0, 23, 30, 37,  0,
        34, 23, 24, 33,  0, 34, 35, 32, 29, 20, 19,  0, 30, 35, 34,  3,  6, 16,
        29, 34, 16, 33, 34, 24, 18,  0, 20, 38, 31, 20, 32, 24, 20, 29, 18, 20,
         1,  6, 16, 29, 34, 16, 33, 34, 24, 18,  0, 20, 38, 31, 20, 32, 24, 20,
        29, 18, 20,  1, 14, 23, 24, 33,  0, 24, 33,  0, 34, 23, 20,  0, 37, 30,
        32, 33, 34,  0, 34, 23, 24, 29, 22,  0, 20, 36, 20, 32,  1, 14, 23, 20,
         0, 20, 36, 20, 29, 34,  0, 33, 34, 16, 32, 34, 33,  0, 16, 34,  0,  4,
         0, 11, 10,  3, 15, 23, 16, 34,  0, 16,  0, 37, 30, 29, 19, 20, 32, 21,
        35, 27,  0, 19, 16, 39,  1,  7, 32, 20, 16, 34,  0, 25, 30, 17,  0, 17,
        39,  0, 34, 23, 20,  0, 34, 20, 16, 28,  1, 15, 23, 16, 34,  0, 16,  0,
        37, 30, 29, 19, 20, 32, 21, 35, 27,  0, 19, 16, 39,  1,  8, 34,  0, 32,
        

* split data into train and val sets

In [7]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [8]:
block_size = 8
train_data[:block_size+1]

tensor([14, 23, 20,  0, 20, 36, 20, 29, 34])

In [9]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target:{target}")

when input is tensor([14]) the target:23
when input is tensor([14, 23]) the target:20
when input is tensor([14, 23, 20]) the target:0
when input is tensor([14, 23, 20,  0]) the target:20
when input is tensor([14, 23, 20,  0, 20]) the target:36
when input is tensor([14, 23, 20,  0, 20, 36]) the target:20
when input is tensor([14, 23, 20,  0, 20, 36, 20]) the target:29
when input is tensor([14, 23, 20,  0, 20, 36, 20, 29]) the target:34


In [10]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size,(batch_size,))
    x = torch.stack([data[i:i+block_size]for i in ix])
    y = torch.stack([data[i+1:i+block_size+1]for i in ix])
    return x,y

xb,yb = get_batch('train')
print('inputs')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('-----')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b,:t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")



inputs
torch.Size([4, 8])
tensor([[ 8,  2, 28,  0, 33, 30,  0, 23],
        [33,  0, 14, 35, 20, 33, 19, 16],
        [24, 18,  0, 20, 38, 31, 20, 32],
        [34,  0, 16,  0, 37, 30, 29, 19]])
targets:
torch.Size([4, 8])
tensor([[ 2, 28,  0, 33, 30,  0, 23, 16],
        [ 0, 14, 35, 20, 33, 19, 16, 39],
        [18,  0, 20, 38, 31, 20, 32, 24],
        [ 0, 16,  0, 37, 30, 29, 19, 20]])
-----
when input is [8] the target: 2
when input is [8, 2] the target: 28
when input is [8, 2, 28] the target: 0
when input is [8, 2, 28, 0] the target: 33
when input is [8, 2, 28, 0, 33] the target: 30
when input is [8, 2, 28, 0, 33, 30] the target: 0
when input is [8, 2, 28, 0, 33, 30, 0] the target: 23
when input is [8, 2, 28, 0, 33, 30, 0, 23] the target: 16
when input is [33] the target: 0
when input is [33, 0] the target: 14
when input is [33, 0, 14] the target: 35
when input is [33, 0, 14, 35] the target: 20
when input is [33, 0, 14, 35, 20] the target: 33
when input is [33, 0, 14, 35, 20, 33] 

In [11]:
print(xb) # input to the transformer

tensor([[ 8,  2, 28,  0, 33, 30,  0, 23],
        [33,  0, 14, 35, 20, 33, 19, 16],
        [24, 18,  0, 20, 38, 31, 20, 32],
        [34,  0, 16,  0, 37, 30, 29, 19]])


In [12]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BiagramLanguageModel(nn.Module):

    def __init__(self,vocab_size):
        super().__init__()

        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)

        ##
        # self.position_embeddings_table = nn.Embedding(block_size,n_embd)
        # self.blocks = nn.Sequential(*[Block(n_embd,n_head=n_head)for _ in range(n_layer)])
        # self.ln_f = nn.LayerNorm(n_embd) # final Layer norm
        # self.lm_head = nn.Linear(n_embd,vocab_size)
        ##

    def forward(self,idx,targets=None):
        logits = self.token_embedding_table(idx)

        ##
        # B,T = idx.shape

        if targets is None:
            loss = None

        else:
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits,targets)

        return logits,loss

    def generate(self,idx,max_new_tokens):
        for _ in range(max_new_tokens):
            logits,loss = self(idx)
            logits = logits[:,-1,:]
            prob = F.softmax(logits,dim=-1)

            idx_next = torch.multinomial(prob,num_samples=1)
            idx = torch.cat((idx,idx_next),dim=1)

        return idx


m = BiagramLanguageModel(vocab_size)
logits,loss = m(xb,yb)
print(logits.shape)
print(loss)


print(decode(m.generate(idx = torch.zeros((1,1),dtype=torch.long),max_new_tokens=100)[0].tolist()))



torch.Size([32, 41])
tensor(3.9208, grad_fn=<NllLossBackward0>)
 tvgRiRvMifG'lgJ'GaMijr!GvynliulmScPPxRgMPr!Sjjdeolbkp'ueAR FsrdmSt dSusa'l aW!exh fswwwwSjMu’tiSfjdm


In [13]:
# Create Pytorch Optimizer
optimizer = torch.optim.AdamW(m.parameters(),lr= 1e-3)

In [14]:
batch_size = 32
for steps in range(100):
    xb,yb = get_batch('train')
    
    #evaluate the loss
    logits,loss = m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

3.859891176223755


In [15]:
print(decode(m.generate(idx=torch.zeros((1,1),dtype = torch.long),max_new_tokens=500)[0].tolist()))

 r5GbTkTjooiknWTFy’MvcS.yFFayjvMPskv'aopig’sT’R.Fywwx5W!ouhaJefellleWTfayvyxiSmkyaAyWsTvA'iTgrpIWTf.lmSueoisp aJPsW!R’5IucbSuWRaMuAiTJd.’JWg5P5eWvcolkJIScJIvgW!WkkJTlInhIpRhWPbydWjiSv’ RxjRiGhejv'lcr’RiPRxfuJutabvMPcllc cwW!JlknA5dproW’ltWWTfertSW!W!fIMPwr!rjdgW!vylcadmsbvIMAR’vgeuJwveStmov'feFgrk'kMFSuedyp WTfpichTfbGyr pluipiyxjp'p tf’uwwrAJb’v!RT’'bpmoiSuSTxnleIvtSj’JS’y5jovvFtPcwM!W’PMokFlAMf pvPnhihe!AJFavrgrwc plvtaJP’ tiyjMoWltMPSjovk!wwPa.’tS'xRgrcW!ScnIkJ!Sfs!!JyuG5Auwpt.p F IPgxmRiWl!FM


* toy example illustrating how matrix multiplication can be used for a "weighted aggregation"

In [16]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
a = a/torch.sum(a,1,keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('b=')
print(b)
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


* Consider the following toy example

In [17]:
torch.manual_seed(1337)
B,T,C = 4,8,2 #batch,time,channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [18]:
# we want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] #(t,c)
        xbow[b,t] = torch.mean(xprev,0)

* version 2 using matrix multiply for a weighted aggregation.

In [19]:
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(1,keepdim=True)
xbow2 = wei @ x # (B,T,T) @ (B,T,C) -----> (B,T,C)
torch.allclose(xbow,xbow2)

False

* version 3 Self Attention

In [20]:
torch.manual_seed(1337)
B,T,C = 4,8,32 #batch,time,channels
x = torch.randn(B,T,C)

# Let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C,head_size,bias=False)
query = nn.Linear(C,head_size,bias=False)
value = nn.Linear(C,head_size,bias=False)
k = key(x)
q = query(x)
wei = q @ k.transpose(-2,-1)


tril = torch.tril(torch.ones(T,T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril==0,float('-inf'))
wei = F.softmax(wei,dim=1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [21]:
wei[0]

tensor([[0.0248, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0052, 0.0091, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0521, 0.0135, 0.2482, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3171, 0.0214, 0.1642, 0.1188, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0412, 0.0487, 0.1046, 0.0742, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1060, 0.5347, 0.2059, 0.1030, 0.7402, 0.0192, 0.0000, 0.0000],
        [0.4298, 0.3409, 0.1769, 0.2027, 0.0480, 0.8472, 0.2329, 0.0000],
        [0.0238, 0.0316, 0.1002, 0.5013, 0.0117, 0.1336, 0.7671, 1.0000]],
       grad_fn=<SelectBackward0>)

In [22]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2,-1) * head_size**-0.5

In [23]:
k.var()

tensor(1.0449)

In [24]:
q.var()

tensor(1.0700)

In [25]:
wei.var()

tensor(1.0918)

In [26]:
torch.softmax(torch.tensor([0.1,-0.2,0.3,-0.2,0.5]),dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [27]:
torch.softmax(torch.tensor([0.1,-0.2,0.3,-0.2,0.5])*8,dim=-1) #gets too peaky,converge to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [28]:
class LayerNorm1d:

    def __init__(self,dim,eps=1e-5,nomentum=0.1):
        self.eps = eps
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)

    def __call__(self,x):
        # calculate the forward pass
        xmean = x.mean(1,keepdim=True) #batch mean
        xvar = x.var(1,keepdim=True) # batch variance
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variacne
        self.out = self.gamma * xhat + self.beta
        return self.out

    def parameters(self):
        return [self.gamma,self.beta]


torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32,100) # barch size 32 of 100 dimensional vectors
x = module(x)
x.shape
        

torch.Size([32, 100])

In [29]:
x[:,0].mean(),x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [30]:
x[0,:].mean(),x[0,:].std()

(tensor(-9.5367e-09), tensor(1.0000))

In [31]:
# import torch
# import torch.nn as nn
# from torch.nn import functional as F

# # hyperparameters
# batch_size = 16 # how many independent sequences will we process in parallel?
# block_size = 32 # what is the maximum context length for predictions?
# max_iters = 5000
# eval_interval = 100
# learning_rate = 1e-3
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# eval_iters = 200
# n_embd = 64
# n_head = 4
# n_layer = 4
# dropout = 0.0
# # ------------

# torch.manual_seed(1337)

# # # wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
# # with open('/content/colab.txt', 'r', encoding='utf-8') as f:
# #     text = f.read()


# # all unique chars that occure in the tweet
# all_text = ''.join(df['tweet'].astype(str))
# chars = sorted(set(all_text))
# vocab_size=len(chars)
# print("Unique Chars")
# print(''.join(chars))
# print('\nToatl unique charactes:',len(chars))



# # here are all the unique characters that occur in this text
# # chars = sorted(list(set(text)))
# # vocab_size = len(chars)
# # # create a mapping from characters to integers
# # stoi = { ch:i for i,ch in enumerate(chars) }
# # itos = { i:ch for i,ch in enumerate(chars) }
# # encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
# # decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# # Train and test splits
# data = torch.tensor(encode(text), dtype=torch.long)
# n = int(0.9*len(data)) # first 90% will be train, rest val
# train_data = data[:n]
# val_data = data[n:]

# # data loading
# def get_batch(split):
#     # generate a small batch of data of inputs x and targets y
#     data = train_data if split == 'train' else val_data
#     ix = torch.randint(len(data) - block_size, (batch_size,))
#     x = torch.stack([data[i:i+block_size] for i in ix])
#     y = torch.stack([data[i+1:i+block_size+1] for i in ix])
#     x, y = x.to(device), y.to(device)
#     return x, y

# @torch.no_grad()
# def estimate_loss():
#     out = {}
#     model.eval()
#     for split in ['train', 'val']:
#         losses = torch.zeros(eval_iters)
#         for k in range(eval_iters):
#             X, Y = get_batch(split)
#             logits, loss = model(X, Y)
#             losses[k] = loss.item()
#         out[split] = losses.mean()
#     model.train()
#     return out

# class Head(nn.Module):
#     """ one head of self-attention """

#     def __init__(self, head_size):
#         super().__init__()
#         self.key = nn.Linear(n_embd, head_size, bias=False)
#         self.query = nn.Linear(n_embd, head_size, bias=False)
#         self.value = nn.Linear(n_embd, head_size, bias=False)
#         self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

#         self.dropout = nn.Dropout(dropout)

#     def forward(self, x):
#         B,T,C = x.shape
#         k = self.key(x)   # (B,T,C)
#         q = self.query(x) # (B,T,C)
#         # compute attention scores ("affinities")
#         wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
#         wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
#         wei = F.softmax(wei, dim=-1) # (B, T, T)
#         wei = self.dropout(wei)
#         # perform the weighted aggregation of the values
#         v = self.value(x) # (B,T,C)
#         out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
#         return out

# class MultiHeadAttention(nn.Module):
#     """ multiple heads of self-attention in parallel """

#     def __init__(self, num_heads, head_size):
#         super().__init__()
#         self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
#         self.proj = nn.Linear(n_embd, n_embd)
#         self.dropout = nn.Dropout(dropout)

#     def forward(self, x):
#         out = torch.cat([h(x) for h in self.heads], dim=-1)
#         out = self.dropout(self.proj(out))
#         return out

# class FeedFoward(nn.Module):
#     """ a simple linear layer followed by a non-linearity """

#     def __init__(self, n_embd):
#         super().__init__()
#         self.net = nn.Sequential(
#             nn.Linear(n_embd, 4 * n_embd),
#             nn.ReLU(),
#             nn.Linear(4 * n_embd, n_embd),
#             nn.Dropout(dropout),
#         )

#     def forward(self, x):
#         return self.net(x)

# class Block(nn.Module):
#     """ Transformer block: communication followed by computation """

#     def __init__(self, n_embd, n_head):
#         # n_embd: embedding dimension, n_head: the number of heads we'd like
#         super().__init__()
#         head_size = n_embd // n_head
#         self.sa = MultiHeadAttention(n_head, head_size)
#         self.ffwd = FeedFoward(n_embd)
#         self.ln1 = nn.LayerNorm(n_embd)
#         self.ln2 = nn.LayerNorm(n_embd)

#     def forward(self, x):
#         x = x + self.sa(self.ln1(x))
#         x = x + self.ffwd(self.ln2(x))
#         return x

# # super simple bigram model
# class BigramLanguageModel(nn.Module):

#     def __init__(self):
#         super().__init__()
#         # each token directly reads off the logits for the next token from a lookup table
#         self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
#         self.position_embedding_table = nn.Embedding(block_size, n_embd)
#         self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
#         self.ln_f = nn.LayerNorm(n_embd) # final layer norm
#         self.lm_head = nn.Linear(n_embd, vocab_size)

#     def forward(self, idx, targets=None):
#         B, T = idx.shape

#         # idx and targets are both (B,T) tensor of integers
#         tok_emb = self.token_embedding_table(idx) # (B,T,C)
#         pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
#         x = tok_emb + pos_emb # (B,T,C)
#         x = self.blocks(x) # (B,T,C)
#         x = self.ln_f(x) # (B,T,C)
#         logits = self.lm_head(x) # (B,T,vocab_size)

#         if targets is None:
#             loss = None
#         else:
#             B, T, C = logits.shape
#             logits = logits.view(B*T, C)
#             targets = targets.view(B*T)
#             loss = F.cross_entropy(logits, targets)

#         return logits, loss

#     def generate(self, idx, max_new_tokens):
#         # idx is (B, T) array of indices in the current context
#         for _ in range(max_new_tokens):
#             # crop idx to the last block_size tokens
#             idx_cond = idx[:, -block_size:]
#             # get the predictions
#             logits, loss = self(idx_cond)
#             # focus only on the last time step
#             logits = logits[:, -1, :] # becomes (B, C)
#             # apply softmax to get probabilities
#             probs = F.softmax(logits, dim=-1) # (B, C)
#             # sample from the distribution
#             idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
#             # append sampled index to the running sequence
#             idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
#         return idx

# model = BigramLanguageModel()
# m = model.to(device)
# # print the number of parameters in the model
# print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# # create a PyTorch optimizer
# optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# for iter in range(max_iters):

#     # every once in a while evaluate the loss on train and val sets
#     if iter % eval_interval == 0 or iter == max_iters - 1:
#         losses = estimate_loss()
#         print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

#     # sample a batch of data
#     xb, yb = get_batch('train')

#     # evaluate the loss
#     logits, loss = model(xb, yb)
#     optimizer.zero_grad(set_to_none=True)
#     loss.backward()
#     optimizer.step()

# # generate from the model
# context = torch.zeros((1, 1), dtype=torch.long, device=device)
# print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))